In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D

# Load MNIST dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

# Buffer size and batch size
BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Generator model
def make_generator_model():
    model = Sequential()
    model.add(Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Output layer with tanh activation
    model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

# Discriminator model
def make_discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(LeakyReLU())
    model.add(Flatten())
    model.add(Dense(1))
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

# Define the loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training loop
EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 16

# Seed for visualization
seed = tf.random.normal([num_examples_to_generate, noise_dim])

def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, epochs):
    for epoch in range(epochs):
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            t = train_step(image_batch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        # Produce images for the GIF as we go
        if (epoch + 1) % 250 == 0:
            generate_and_save_images(generator, epoch + 1, seed)

        # Save the model every 50 epochs
        if (epoch + 1) % 50 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

    return gen_loss_list, disc_loss_list

# Generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

gen_loss_list, disc_loss_list = train(train_dataset, EPOCHS)

# Plotting the training losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(gen_loss_list,label="G")
plt.plot(disc_loss_list,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf

# Load the CranexD dataset
dataset = tf.keras.datasets.cranexd
(x_train, y_train), (x_test, y_test) = dataset.load_data()

# Define the generator and discriminator networks
generator = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(224, 224, 3)),
  tf.keras.layers.Conv2D(64, (3, 3), padding=’same’),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, (3, 3), padding=’same’),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1024),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.Dense(224 * 224 * 3),
  tf.keras.layers.Reshape((224, 224, 3))
])

discriminator = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(224, 224, 3)),
  tf.keras.layers.Conv2D(64, (3, 3), padding=’same’),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, (3, 3), padding=’same’),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1024),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.Dense(1, activation=’sigmoid’)
])

# Compile the generator and discriminator networks
generator.compile(optimizer=’adam’, loss=’binary_crossentropy’, metrics=[‘accuracy’])
discriminator.compile(optimizer=’adam’, loss=’binary_crossentropy’, metrics=[‘accuracy’])

# Define the GAN loss
gan_loss = tf.keras.losses.BinaryCrossentropy()

# Define the GAN training steps
@tf.function
def train_step(x_batch):
  # Generate fake images
  fake_images = generator(x_batch)

  # Train the discriminator
  real_labels = tf.ones((x_batch.shape[0], 1))
  fake_labels = tf.zeros((x_batch.shape[0], 1))
  discriminator_loss = discriminator.train_on_batch([x_batch, fake_images], [real_labels, fake_labels])

  # Train the generator
  generator_loss = gan_loss(tf.ones((x_batch.shape[0], 1)), generator(x_batch))

  return discriminator_loss, generator_loss

# Train the GAN
epochs = 10
for epoch in range(epochs):
  discriminator_loss, generator_loss = train_step(x_train)
  print(f’Epoch {epoch + 1}/{epochs}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}’)

# Generate fake images
fake_images = generator.predict(x_test)

# Save the generated images
tf.keras.utils.save_img(fake_images, ‘generated_images.jpg’)
